Needed imports and utility

In [ ]:
import os
import ansys.speos.core as core
import ansys.speos.workflow as workflow

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

def print_message(msg):
    print(core.protobuf_message_to_str(msg))

# Create connection with speos rpc server

In [ ]:
from ansys.speos.core.speos import Speos
speos = Speos(host="localhost", port=50051)
workflow.clean_all_dbs(speos.client)

# Create SpeosSimulationUpdate

In [ ]:
simu_name = "Inverse_SeveralSensors.speos"
speos_file = os.path.join(tests_data_path, os.path.join(simu_name, simu_name))

ssu = workflow.SpeosSimulationUpdate(speos=speos, file_name=speos_file)
print(ssu.status)
print(ssu.scene)

# New Cameras
## Create two PhotometricCameraSensorParameters

In [ ]:
camera_input_path = os.path.join(tests_data_path, "CameraInputFiles")

camera_fov_190 = workflow.PhotometricCameraSensorParameters()
camera_fov_190.name = "FOV_190deg"
camera_fov_190.transmittance_file = os.path.join(camera_input_path, "CameraTransmittance.spectrum")
camera_fov_190.distorsion_file = os.path.join(camera_input_path, "CameraDistortion_190deg.OPTDistortion")
camera_fov_190.color_mode_red_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityRed.spectrum")
camera_fov_190.color_mode_green_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityGreen.spectrum")
camera_fov_190.color_mode_blue_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityBlue.spectrum")
camera_fov_190.wavelengths_start = 380
camera_fov_190.wavelengths_end = 780
camera_fov_190.wavelengths_sampling = 21
camera_fov_190.focal_length = 0.9
camera_fov_190.imager_distance = 19.1
camera_fov_190.f_number = 2
camera_fov_190.horizontal_pixel = 640
camera_fov_190.vertical_pixel = 480
camera_fov_190.width = 3.6
camera_fov_190.height = 2.7

camera_fov_150 = camera_fov_190.copy()
camera_fov_150.name = "FOV_150deg"
camera_fov_150.distorsion_file = os.path.join(camera_input_path, "CameraDistortion_150deg.OPTDistortion")

cameras = []
cameras.append(camera_fov_190)
cameras.append(camera_fov_150)


## Create CameraSensorProperties

In [ ]:
camera_props = workflow.CameraSensorProperties()
camera_props.origin = [17, 10, 15]
camera_props.x_vector = [0.0, 0.0, -1.0]
camera_props.y_vector = [0.0, 1.0, 0.0]
camera_props.z_vector = [1.0, 0.0, 0.0]

# Add cameras to the simulation

In [ ]:
for camera in cameras:
    ssu.add_camera_sensor(camera, camera_props)

# Compute

In [ ]:
job_link = ssu.compute("first_job")
print_message(job_link.get_results())

# Clean after usage

In [ ]:
job_link.delete()

In [ ]:
ssu.close()
print(ssu.status)